<h1 style='text-align:center'>Convolutional Neural Networks</h1>

### What are CNNs used for?

In [1]:
from IPython.display import IFrame
IFrame(src="https://www.youtube.com/embed/ACmydtFDTGs", width='622', height='466')

CNNs are mainly used for image recognition/classification. They can be used for video analysis, NLP (sentiment analysis, topic modeling), and speech recognition. Today we'll be discussing how CNNs are you used to classify images. 

### How do our brains see an image? 


We might see some fluffy tail, a wet nose, flappy ears, and a good boy and conclude we are probably seeing dog. There is not one singular thing about a dog that our brain recognizes as a dog but an amalgamation of different patterns that allow us to make a probable guess.  


<img src='./img/chihuahua.jpeg'/>

### Convolutal Neural Networks - How computers see images

<img src='./img/architecture.jpeg'/>

###### Images

To computers images are a 3D object - composed of 3 matrices - one for each primary color that can be combined in varying intensities to create different colors. Each element in a matrix represents the location of a pixel and contains a number between 0 and 255 which indicates the intensity of the corresponding primary color in that pixel. 

##### From wikipedia:

    "The RGB color model is an additive color model[1] in which red, green and blue light are added together in various ways to reproduce a broad array of colors. The name of the model comes from the initials of the three additive primary colors, red, green, and blue. The main purpose of the RGB color model is for the sensing, representation, and display of images in electronic systems, such as televisions and computers"

<img src='./img/rgb.png'/>

### Why do we want CNNs

![](./img/NN_vs_CNN.png)

![](./img/weight_sharing_1.png)

### Convolution

<b>To Convolve means to roll together</b><br/><br/>
In the context of mathematics think of it as a way of mixing two functions by multiplying them.<br/><br/>

Convolutional neural networks make use of linear algebra to identify patterns using the pixel values (intensity of R,G, or B). By taking a small matrix and moving it across an image and multiplying them together every time it moves our network can mathematically identify patterns in these images. This small matrix is known as a <b>"kernel"</b> or <b>"filter"</b> and each one is designed to identify a particular pattern in an image (edges, shapes, etc.)

<img src='./img/convolve.gif' width='300px'/>

![](./img/conv.gif)

When a filter is "rolled over" an image the resulting matrix is called a <b>Feature Map</b> - literally a map of where each pattern of feature is in the image. Elements with higher values indicate the presence of that pattern the filter is looking. 

The values (or weights) of the filter are adjusted during back-propagation.

Every node in a neural network layer is a different filter looking for a certain pattern. 

##### Typically use 3X3 maps

##### For N⨉N input and kernel size k⨉k the output size is M = N - k + 1

<h3>Parameters</h4>

<b>Padding</b> - Sometimes it is convenient to pad the input volume with zeros around the border. Helps with detecting patterns at the edge of an image. 

- Padding 'SAME' - keeps image the same size (k-1/2 on each side for map sized k) 
- Padding 'VALID' - padding is 0.

Padding is used to control resolution in the output layer!

<b>Stride</b> - The number of pixels to shift the filter on each "roll". The larger the stride the smaller the feature map will be - but we will lose more information.

Used for:
 - faster processsing
 - invariance to translation
 
$Output = \frac{I + 2pad - map}{stride} + 1$

### Pooling

The feature maps are fed into a max pool layer, and like convolutions, this method is applied one patch at a time (usually 2x2). Max pooling simply takes the largest value from one patch of an image, places it in a new matrix next to the max values from other patches, and discards the rest of the information contained in the activation maps. Other methods exist such as average pooling (taking an average of the patch). 

<img src='./img/maxpool.png'/>


This process results in a new feature map with reduced dimensionality that is then passed into another convolution layer to continue the pattern finding process. These steps are repeated until they are passed to a fully connected layer that proceeds to classify the image using the identified patterns. 

Used for:
    - invarince for translation
    - faster processing 

### Dilation

Use for high resolution to reduce complexity

$Output = \frac{I + 2pad - map'}{stride} + 1$

$map' = map + (map-1)*(dilate - 1)$

### Classification

Once the neural networks has collected a series of patterns that an image contains it is ready to make a guess as to what the image is. In order to do so, it starts by flattening the 2D matrix into a 1D vector, so it can be passed into a normal densely connected layer for classification. Then using this vector one or many densely connected layers will make a prediction as to what the image is. 

<img src="img/typical_CNN.png" width=150>

### Transfer Learning

Transfer learning is the process of training the last few layers of a pre-trained model to make classifications for a specific use case. This is not exclusive to CNNs but can be applied other neural networks and ML models. Pre-trained CNN models have dozens to hundreds of layers that have been trained on millions of images. Through this training they are very good at identifying complex patterns in image and these collection of patterns can then be used to train the final densely connected layers to make a specific classification. Examples of these pre-trained models include ResNet, InceptionNet, VGG16 - many of which are available through Keras. 

### Code 

In [2]:
import os

import matplotlib.pyplot as plt
import numpy as np
import keras
from keras import models
from keras import layers
from sklearn.metrics import confusion_matrix, f1_score
from keras.models import load_model
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

np.random.seed(123)

Using TensorFlow backend.


PNG files are transparent layers 

## Load Training Data

In [3]:
# download from https://www.kaggle.com/dansbecker/hot-dog-not-hot-dog/download
!unzip data/hot-dog-not-hot-dog.zip -d data/

Archive:  data/hot-dog-not-hot-dog.zip
  inflating: data/seefood/test/hot_dog/133012.jpg  
  inflating: data/seefood/test/hot_dog/133015.jpg  
  inflating: data/seefood/test/hot_dog/133245.jpg  
  inflating: data/seefood/test/hot_dog/135628.jpg  
  inflating: data/seefood/test/hot_dog/138933.jpg  
  inflating: data/seefood/test/hot_dog/138937.jpg  
  inflating: data/seefood/test/hot_dog/138969.jpg  
  inflating: data/seefood/test/hot_dog/139177.jpg  
  inflating: data/seefood/test/hot_dog/142835.jpg  
  inflating: data/seefood/test/hot_dog/143012.jpg  
  inflating: data/seefood/test/hot_dog/144956.jpg  
  inflating: data/seefood/test/hot_dog/146227.jpg  
  inflating: data/seefood/test/hot_dog/146834.jpg  
  inflating: data/seefood/test/hot_dog/147874.jpg  
  inflating: data/seefood/test/hot_dog/152106.jpg  
  inflating: data/seefood/test/hot_dog/154409.jpg  
  inflating: data/seefood/test/hot_dog/154545.jpg  
  inflating: data/seefood/test/hot_dog/156126.jpg  
  inflating: data/seefood

  inflating: data/seefood/test/hot_dog/554307.jpg  
  inflating: data/seefood/test/hot_dog/558890.jpg  
  inflating: data/seefood/test/hot_dog/561681.jpg  
  inflating: data/seefood/test/hot_dog/564558.jpg  
  inflating: data/seefood/test/hot_dog/566784.jpg  
  inflating: data/seefood/test/hot_dog/566965.jpg  
  inflating: data/seefood/test/hot_dog/570799.jpg  
  inflating: data/seefood/test/hot_dog/571081.jpg  
  inflating: data/seefood/test/hot_dog/573731.jpg  
  inflating: data/seefood/test/hot_dog/574379.jpg  
  inflating: data/seefood/test/hot_dog/576265.jpg  
  inflating: data/seefood/test/hot_dog/580378.jpg  
  inflating: data/seefood/test/hot_dog/581166.jpg  
  inflating: data/seefood/test/hot_dog/583811.jpg  
  inflating: data/seefood/test/hot_dog/588881.jpg  
  inflating: data/seefood/test/hot_dog/591179.jpg  
  inflating: data/seefood/test/hot_dog/593867.jpg  
  inflating: data/seefood/test/hot_dog/595636.jpg  
  inflating: data/seefood/test/hot_dog/595968.jpg  
  inflating:

  inflating: data/seefood/test/not_hot_dog/38337.jpg  
  inflating: data/seefood/test/not_hot_dog/38349.jpg  
  inflating: data/seefood/test/not_hot_dog/38442.jpg  
  inflating: data/seefood/test/not_hot_dog/38615.jpg  
  inflating: data/seefood/test/not_hot_dog/38768.jpg  
  inflating: data/seefood/test/not_hot_dog/39461.jpg  
  inflating: data/seefood/test/not_hot_dog/40094.jpg  
  inflating: data/seefood/test/not_hot_dog/40231.jpg  
  inflating: data/seefood/test/not_hot_dog/40449.jpg  
  inflating: data/seefood/test/not_hot_dog/40762.jpg  
  inflating: data/seefood/test/not_hot_dog/40944.jpg  
  inflating: data/seefood/test/not_hot_dog/40947.jpg  
  inflating: data/seefood/test/not_hot_dog/41693.jpg  
  inflating: data/seefood/test/not_hot_dog/42125.jpg  
  inflating: data/seefood/test/not_hot_dog/42252.jpg  
  inflating: data/seefood/test/not_hot_dog/42487.jpg  
  inflating: data/seefood/test/not_hot_dog/42609.jpg  
  inflating: data/seefood/test/not_hot_dog/42954.jpg  
  inflatin

  inflating: data/seefood/test/not_hot_dog/85375.jpg  
  inflating: data/seefood/test/not_hot_dog/85398.jpg  
  inflating: data/seefood/test/not_hot_dog/85452.jpg  
  inflating: data/seefood/test/not_hot_dog/85514.jpg  
  inflating: data/seefood/test/not_hot_dog/86180.jpg  
  inflating: data/seefood/test/not_hot_dog/86528.jpg  
  inflating: data/seefood/test/not_hot_dog/86782.jpg  
  inflating: data/seefood/test/not_hot_dog/86812.jpg  
  inflating: data/seefood/test/not_hot_dog/87020.jpg  
  inflating: data/seefood/test/not_hot_dog/87125.jpg  
  inflating: data/seefood/test/not_hot_dog/87246.jpg  
  inflating: data/seefood/test/not_hot_dog/88158.jpg  
  inflating: data/seefood/test/not_hot_dog/89027.jpg  
  inflating: data/seefood/test/not_hot_dog/8917.jpg  
  inflating: data/seefood/test/not_hot_dog/8984.jpg  
  inflating: data/seefood/test/not_hot_dog/9053.jpg  
  inflating: data/seefood/test/not_hot_dog/9547.jpg  
  inflating: data/seefood/test/not_hot_dog/9555.jpg  
  inflating: da

  inflating: data/seefood/train/hot_dog/3701301.jpg  
  inflating: data/seefood/train/hot_dog/3711230.jpg  
  inflating: data/seefood/train/hot_dog/3711747.jpg  
  inflating: data/seefood/train/hot_dog/3714525.jpg  
  inflating: data/seefood/train/hot_dog/3716190.jpg  
  inflating: data/seefood/train/hot_dog/3724744.jpg  
  inflating: data/seefood/train/hot_dog/3726090.jpg  
  inflating: data/seefood/train/hot_dog/3726741.jpg  
  inflating: data/seefood/train/hot_dog/3728106.jpg  
  inflating: data/seefood/train/hot_dog/3730066.jpg  
  inflating: data/seefood/train/hot_dog/3733563.jpg  
  inflating: data/seefood/train/hot_dog/3733809.jpg  
  inflating: data/seefood/train/hot_dog/3736382.jpg  
  inflating: data/seefood/train/hot_dog/3742819.jpg  
  inflating: data/seefood/train/hot_dog/3742960.jpg  
  inflating: data/seefood/train/hot_dog/3746979.jpg  
  inflating: data/seefood/train/hot_dog/3748381.jpg  
  inflating: data/seefood/train/hot_dog/3750744.jpg  
  inflating: data/seefood/tr

...

  inflating: data/test/hot_dog/558890.jpg  
  inflating: data/test/hot_dog/561681.jpg  
  inflating: data/test/hot_dog/564558.jpg  
  inflating: data/test/hot_dog/566784.jpg  
  inflating: data/test/hot_dog/566965.jpg  
  inflating: data/test/hot_dog/570799.jpg  
  inflating: data/test/hot_dog/571081.jpg  
  inflating: data/test/hot_dog/573731.jpg  
  inflating: data/test/hot_dog/574379.jpg  
  inflating: data/test/hot_dog/576265.jpg  
  inflating: data/test/hot_dog/580378.jpg  
  inflating: data/test/hot_dog/581166.jpg  
  inflating: data/test/hot_dog/583811.jpg  
  inflating: data/test/hot_dog/588881.jpg  
  inflating: data/test/hot_dog/591179.jpg  
  inflating: data/test/hot_dog/593867.jpg  
  inflating: data/test/hot_dog/595636.jpg  
  inflating: data/test/hot_dog/595968.jpg  
  inflating: data/test/hot_dog/599728.jpg  
  inflating: data/test/hot_dog/602204.jpg  
  inflating: data/test/hot_dog/604245.jpg  
  inflating: data/test/hot_dog/607287.jpg  
  inflating: data/test/hot_dog/6

  inflating: data/test/not_hot_dog/45896.jpg  
  inflating: data/test/not_hot_dog/46246.jpg  
  inflating: data/test/not_hot_dog/46325.jpg  
  inflating: data/test/not_hot_dog/46401.jpg  
  inflating: data/test/not_hot_dog/47115.jpg  
  inflating: data/test/not_hot_dog/47388.jpg  
  inflating: data/test/not_hot_dog/47909.jpg  
  inflating: data/test/not_hot_dog/48052.jpg  
  inflating: data/test/not_hot_dog/48073.jpg  
  inflating: data/test/not_hot_dog/48151.jpg  
  inflating: data/test/not_hot_dog/48208.jpg  
  inflating: data/test/not_hot_dog/48506.jpg  
  inflating: data/test/not_hot_dog/48840.jpg  
  inflating: data/test/not_hot_dog/49803.jpg  
  inflating: data/test/not_hot_dog/50193.jpg  
  inflating: data/test/not_hot_dog/50277.jpg  
  inflating: data/test/not_hot_dog/51368.jpg  
  inflating: data/test/not_hot_dog/51481.jpg  
  inflating: data/test/not_hot_dog/51533.jpg  
  inflating: data/test/not_hot_dog/51678.jpg  
  inflating: data/test/not_hot_dog/52510.jpg  
  inflating: 

  inflating: data/train/hot_dog/1190833.jpg  
  inflating: data/train/hot_dog/1192908.jpg  
  inflating: data/train/hot_dog/1195487.jpg  
  inflating: data/train/hot_dog/1195698.jpg  
  inflating: data/train/hot_dog/1195977.jpg  
  inflating: data/train/hot_dog/1204467.jpg  
  inflating: data/train/hot_dog/120471.jpg  
  inflating: data/train/hot_dog/1205205.jpg  
  inflating: data/train/hot_dog/1207811.jpg  
  inflating: data/train/hot_dog/1210977.jpg  
  inflating: data/train/hot_dog/1211682.jpg  
  inflating: data/train/hot_dog/122007.jpg  
  inflating: data/train/hot_dog/1224013.jpg  
  inflating: data/train/hot_dog/122606.jpg  
  inflating: data/train/hot_dog/1226430.jpg  
  inflating: data/train/hot_dog/1226791.jpg  
  inflating: data/train/hot_dog/1233688.jpg  
  inflating: data/train/hot_dog/1234491.jpg  
  inflating: data/train/hot_dog/1236871.jpg  
  inflating: data/train/hot_dog/124323.jpg  
  inflating: data/train/hot_dog/1245977.jpg  
  inflating: data/train/hot_dog/124987

  inflating: data/train/hot_dog/987144.jpg  
  inflating: data/train/hot_dog/987193.jpg  
  inflating: data/train/hot_dog/996310.jpg  
  inflating: data/train/not_hot_dog/100135.jpg  
  inflating: data/train/not_hot_dog/100148.jpg  
  inflating: data/train/not_hot_dog/100274.jpg  
  inflating: data/train/not_hot_dog/100945.jpg  
  inflating: data/train/not_hot_dog/101312.jpg  
  inflating: data/train/not_hot_dog/102037.jpg  
  inflating: data/train/not_hot_dog/102197.jpg  
  inflating: data/train/not_hot_dog/102813.jpg  
  inflating: data/train/not_hot_dog/103010.jpg  
  inflating: data/train/not_hot_dog/105375.jpg  
  inflating: data/train/not_hot_dog/105647.jpg  
  inflating: data/train/not_hot_dog/106608.jpg  
  inflating: data/train/not_hot_dog/107013.jpg  
  inflating: data/train/not_hot_dog/107216.jpg  
  inflating: data/train/not_hot_dog/107312.jpg  
  inflating: data/train/not_hot_dog/107816.jpg  
  inflating: data/train/not_hot_dog/108310.jpg  
  inflating: data/train/not_hot_

  inflating: data/train/not_hot_dog/311779.jpg  
  inflating: data/train/not_hot_dog/319830.jpg  
  inflating: data/train/not_hot_dog/320734.jpg  
  inflating: data/train/not_hot_dog/322067.jpg  
  inflating: data/train/not_hot_dog/331674.jpg  
  inflating: data/train/not_hot_dog/333385.jpg  
  inflating: data/train/not_hot_dog/338101.jpg  
  inflating: data/train/not_hot_dog/4176.jpg  
  inflating: data/train/not_hot_dog/4770.jpg  
  inflating: data/train/not_hot_dog/4781.jpg  
  inflating: data/train/not_hot_dog/4889.jpg  
  inflating: data/train/not_hot_dog/5762.jpg  
  inflating: data/train/not_hot_dog/5764.jpg  
  inflating: data/train/not_hot_dog/5813.jpg  
  inflating: data/train/not_hot_dog/6021.jpg  
  inflating: data/train/not_hot_dog/6127.jpg  
  inflating: data/train/not_hot_dog/748088.jpg  
  inflating: data/train/not_hot_dog/749374.jpg  
  inflating: data/train/not_hot_dog/759160.jpg  
  inflating: data/train/not_hot_dog/765053.jpg  
  inflating: data/train/not_hot_dog/76

In [4]:
!open data

## Preprocess Images

In [5]:
# get all the data in the directory split/test, and reshape them
data_te = ImageDataGenerator(rescale=1./255).flow_from_directory(
    'data/test',
    target_size=(224, 224),
    batch_size=4419,
    seed=123)

Found 500 images belonging to 2 classes.


In [ ]:
data_tr = ImageDataGenerator(rescale=1./255).flow_from_directory(
    'data/train',
    target_size=(224, 224),
    batch_size=4419,
    seed=123)

In [ ]:
#split images and labels
images_tr, labels_tr = next(data_tr)

In [ ]:
images_tr.shape

In [ ]:
#split images and labels
images_te, labels_te = next(data_te)

In [ ]:
images = np.concatenate((images_tr, images_te))

In [ ]:
labels = np.concatenate((labels_tr[:,0], labels_te[:,0]))

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.imshow(images[28])
plt.show()

## 3. Convolutional neural network

In [ ]:
from sklearn.model_selection import train_test_split
X_model, X_test, y_model, y_test = train_test_split(images, labels, test_size=0.20, random_state=123)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X_model, y_model, test_size=0.20, random_state=123)

In [ ]:
cnn = models.Sequential()
cnn.add(layers.Conv2D(64, (3, 3),
                      activation='relu',
                      input_shape=(224, 224,  3),
                      padding='SAME'))
# 64 bias parameters
# 64 * (3 * 3 * 3) weight parametrs
# Output is 64*224*224
cnn.add(layers.MaxPooling2D((2, 2)))
# Output is 64*112*112
cnn.add(layers.Conv2D(32, (3, 3), activation='relu'))
# 32 bias parameters
# 32 * (3*3*64)
# Output is 32*112*112
cnn.add(layers.MaxPooling2D((2, 2)))
cnn.add(layers.Flatten())
cnn.add(layers.Dense(32, activation='relu'))
cnn.add(layers.Dense(1, activation='sigmoid'))

cnn.compile(loss='binary_crossentropy',
            optimizer="sgd",
            metrics=['acc'])

In [ ]:
saving_weights = keras.callbacks.ModelCheckpoint(
    'weights.{epoch:02d}-{val_loss:.2f}.hdf5',
    monitor='val_loss', verbose=0, save_best_only=False,
    save_weights_only=False, mode='auto', period=10
)

In [ ]:
reduce_lr = keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss', factor=0.1, patience=20,
    verbose=0, mode='auto', min_delta=0.0001, min_lr=0)

In [ ]:
nan_problem = keras.callbacks.TerminateOnNaN()

In [ ]:
early_stop = keras.callbacks.EarlyStopping(
    monitor='val_loss', min_delta=0, patience=20,
    verbose=0, mode='auto', baseline=None, restore_best_weights=False)


In [ ]:
csv_logger = keras.callbacks.CSVLogger('training.log')


In [ ]:
print(cnn.summary())

In [ ]:
cnn1 = cnn.fit(X_train,
               y_train,
               epochs=2,
               batch_size=200,
               validation_data=(X_val, y_val),
               callbacks=[csv_logger, early_stop, nan_problem, reduce_lr, saving_weights])

In [ ]:
hist_cnn = cnn1.history
loss_values = hist_cnn['loss']
val_loss_values = hist_cnn['val_loss']
acc_values = hist_cnn['acc']
val_acc_values = hist_cnn['val_acc']


epochs = range(1, len(loss_values) + 1)

plt.figure(figsize=(15, 4))
plt.subplot(121)
plt.plot(epochs, loss_values, 'g.', label='Training loss')
plt.plot(epochs, val_loss_values, 'g', label='Validation loss')

plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.subplot(122)
plt.plot(epochs, acc_values, 'r.', label='Training acc')
plt.plot(epochs, val_acc_values, 'r', label='Validation acc')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

In [ ]:
results_train = cnn.evaluate(X_train, y_train)
results_test = cnn.evaluate(X_test, y_test)
print(results_train, results_test)

In [ ]:
# This should load a weight file in, but I can't seem to figure out how to make this work
# cnn.load_weights("weights.20-0.75.hdf5")

In [ ]:
# cnn.evaluate(X_val, y_val)

## Transfer Learning

### InceptionNet

In [ ]:
from keras.applications import inception_v3
from keras.layers import Dense, GlobalAveragePooling2D
from sklearn.metrics import confusion_matrix, f1_score
from keras.models import Model

In [ ]:
imagenet = inception_v3.InceptionV3(weights='imagenet', include_top=False)
imagenet_new = imagenet.output
new_model = models.Sequential()
new_model.add(imagenet)
new_model.add(GlobalAveragePooling2D())
new_model.add(Dense(1024, activation='relu'))
new_model.add(Dense(1024, activation='relu'))  # dense layer 2
new_model.add(Dense(512, activation='relu'))  # dense layer 3
# final layer with softmax activation
new_model.add(Dense(1, activation='sigmoid'))

In [ ]:
for i, layer in enumerate(imagenet.layers):
    print(i, layer.name, layer.trainable)

In [ ]:
for i,layer in enumerate(new_model.layers):
    print(i,layer.name, layer.trainable)

In [ ]:
for layer in new_model.layers[:1]:
    layer.trainable=False

In [ ]:
for i,layer in enumerate(new_model.layers):
    print(i,layer.name, layer.trainable)

In [ ]:
new_model.compile(optimizer='Adam',
                  loss='binary_crossentropy', metrics=['accuracy'])

# step_size_train=train_generator.n//train_generator.batch_size
new_model.fit(X_train,
              y_train,
              epochs=10,
              batch_size=50,
              validation_data=(X_val, y_val))

In [ ]:
new_model.evaluate(X_test, y_test)

In [ ]:
predictions_transfer = new_model.predict(X_test)
predictions_transfer = np.around(predictions_transfer)

In [ ]:
f1_score(y_test, predictions_transfer)

In [ ]:
import itertools

def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.tight_layout()

In [ ]:
plt.figure()
plot_confusion_matrix(confusion_matrix(y_test, predictions_transfer), classes=['hot dog', 'not hot dog'], normalize=False,
                      title='Confusion matrix - ImagenetV3')

In [ ]:
def predictoneimage_cnn(model, path):
    img = load_img(path, target_size=(224, 224))
    plt.imshow(img)
    img = img_to_array(img)
    img = img/255
    img = np.expand_dims(img, axis=0)
    predict = model.predict(img)
    return predict

In [ ]:
predictoneimage_cnn(new_model, '/content/hot-dog-not-hot-dog/test/not_hot_dog/13719.jpg')

##### Resources

https://skymind.ai/wiki/convolutional-network

https://cs231n.github.io/convolutional-networks/

